# Evaluación de modelos (ejercicio)

### **Importante: comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría.

Para este ejercicio usaremos de nuevo los datos de cáncer de mama que están accesibles en scikit-learn.

In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

## Parte 1: validación cruzada estratificada

Se pide implementar una funcion `mi_cross_val_f1` que aplique validación cruzada con $k$ particiones estratificadas, y que use **$F_1$ como métrica**. Similar a la función `cross_val_score`, pero específicamente con particiones estratificadas y métrica $F_1$, **y obviamente sin usar `cross_val_score`**.  

En concreto, la función debe tener los siguientes argumentos de entrada, `mi_cross_val_f1`*(modelo,X,y,n_splits=5,clase_positiva=1)*, donde:

- *modelo* es un un clasificador (en particular, un objeto de una clase que tenga métodos `fit` y `predict`).
- *X* es un array con los datos.
- *y* es un array con la clasificación (binaria) de los datos anteriores.
- *n_splits* es el número de particiones en la validación cruzada. Por defecto es 5.
- *clase_positiva* la clase que se ha considerar como positiva cuando se calcule la métrica $F_1$. Por defecto es 1. Nótese que al calcular la métrica $F_1$ debemos saber qué clase es la que se considera como positiva.  

Debe devolver una lista o array con *n_splits* números, los resultados de las *n_splits* evaluaciones de la métrica $F_1$ que se realizan al aplicar validación cruzada con *n_splits* particiones estratificadas. 

**Nota**: no se puede usar `cross_val_score`, pero se debe usar `f1_score`y la clase `StratifiedKFold` (consultar la documentación de ambas)

**Ejemplos**:

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify=cancer.target, random_state=0)

from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(solver="liblinear")


# mi_cross_val_f1(logreg,X_train,y_train) devuelve:

#[0.9629629629629629,
# 0.9541284403669724,
# 0.9320388349514563,
# 0.9814814814814815,
# 0.9724770642201834]

# mi_cross_val_f1(logreg,X_train,y_train,n_splits=10) devuelve:

#[0.9818181818181818,
# 0.9433962264150944,
# 0.9433962264150944,
# 0.9642857142857143,
# 0.9433962264150944,
# 0.9629629629629629,
# 0.9818181818181818,
# 0.9811320754716981,
# 0.9811320754716981,
# 0.962962962962963]

# mi_cross_val_f1(logreg,X_train,y_train,n_splits=10,clase=0)

#[0.967741935483871,
# 0.9090909090909091,
# 0.9090909090909091,
# 0.9333333333333333,
# 0.9090909090909091,
# 0.9375,
# 0.967741935483871,
# 0.967741935483871,
# 0.967741935483871,
# 0.9285714285714286]

In [3]:
# Definr aquí mi_cross_val_f1

Se puede comprobar que los tres ejemplos anteriores efectivamente coinciden con lo que devuelve `cross_val_score`:

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer,f1_score

In [5]:
cross_val_score(logreg,X_train,y_train,scoring=make_scorer(f1_score),cv=5)

array([0.96296296, 0.95412844, 0.93203883, 0.98148148, 0.97247706])

In [6]:
cross_val_score(logreg,X_train,y_train,scoring=make_scorer(f1_score),cv=10)

array([0.98181818, 0.94339623, 0.94339623, 0.96428571, 0.94339623,
       0.96296296, 0.98181818, 0.98113208, 0.98113208, 0.96296296])

In [7]:
cross_val_score(logreg,X_train,y_train,scoring=make_scorer(f1_score,pos_label=0),cv=10)

array([0.96774194, 0.90909091, 0.90909091, 0.93333333, 0.90909091,
       0.9375    , 0.96774194, 0.96774194, 0.96774194, 0.92857143])

## Parte 2: evaluación de modelos para datos del cáncer de mama

En esta parte supondremos que queremos encontrar un buen modelo para clasificación en el caso del cáncer de mama. Nos restingiremos a dos familias de clasificadores:

1. **SGDClassifier** con *loss="log"*. Es decir, regresión logística mediante descenso por el gradiente. 
2. **SVC**, máquina de vectores soporte con kernels.

En los datos del cáncer de mama, la clase 0 se corresponde con *maligno* y la clase 1 se corresponde con *benigno*. Supondremos también en esta sección que estamos interesado principalmente en el rendimiento sobre la detección del cáncer. Es decir, consideremos *maligno* como la clase positiva.

**Nota importante**: la clase positiva por defecto en scikit-learn es 1, y en este caso queremos considerar la clase 0 como la positiva. Para ello tenemos dos opciones:

- Usar el parámetro `pos_label` en las distintas funciones que usemos.
- Hacer una transformación inicial de las clases, para poner el 1 como 0 y el 0 como 1. Eso se consigue fácilmente haciendo `1-cancer.target`

### 2.1 *Grid Search* con validación cruzada



Aplicar `GridSearchCV` para ajustar hiperparámetros de los modelos `SGDClassifier`y `SVC`, aplicados al conjunto de datos del cáncer de mama.  En lugar de tratar de optimizar la tasa de aciertos, tratar de optimizar la métrica $F_1$ calculada considerando que los ejemplos de tumor "maligno" son los positivos. 

Tratar de encontrar valores optimos solo para aquellos hiperparámetos del modelo que te sean familiares y para valores de los que conozcas el significado. Probar con distintas rejillas. 

Una vez ajustados los dos modelos, dar para ambos:

- La tasa de aciertos, tanto en el conjunto de entrenamiento como en el de prueba.
- Matriz de confusión, tanto en entrenamiento como en prueba.
- *Classification report*, tanto para entrenamiento como para prueba. 

Interpretar los resultados. 


### 2.2 Curvas *ROC* y *PR*

Comparar ambos modelos construidos en el apartado anterior, dibujando sendas gráficas con las curvas ROC de ambos y con las curvas PR de ambos. Interretar las gráficas y calcular para ambos modelos la precisión media (*average precision*) y el área bajo la curva (*AUC*).

Interpretar los resultados.

In [8]:
import matplotlib.pyplot as plt